# Podstawowe predykaty geometryczne, przeprowadzanie testów, wizualizacja i opracowanie wyników

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pathlib
from labs.tests.test1 import Test
from labs.visualizer.main import Visualizer

# Przydatne funkcje 

In [ ]:
def draw_points(points, title = ""):
    vis = Visualizer()
    vis.add_point(points, s=30, color='green')
    if title :
        target_dir = pathlib.Path("Imgs/Other")
        target_dir.mkdir(parents=True, exist_ok=True)
        vis.save(target_dir / f"{title}.png")
    vis.show()

In [ ]:
def draw_line(points_left, points_mid, points_right, title=""):
    vis = Visualizer()
    vis.add_line(((-1.0, 0.0), (1.0, 0.1)), color='black')
    points_and_their_colors = [
        (len(points_left), points_left, 'green'),
        (len(points_mid), points_mid, 'purple'),
        (len(points_right), points_right, 'orange')
    ]

    points_and_their_colors.sort(reverse=True, key=lambda x: x[0])
    for length, points, color in points_and_their_colors:
        vis.add_point(points, s=30, color=color)

    vis.add_title(title)
    vis.show()
    if title :
        folder_label = title.split()[0].strip()
        temp_title = title.replace(':', ' ').replace(',', ' ').replace('=', ' ')
        safe_title = temp_title.replace(' ', '_')
        target_dir = pathlib.Path('Imgs') / folder_label

        target_dir.mkdir(parents=True, exist_ok=True)

        filename = f"{safe_title}.png"
        full_path = target_dir / filename
        vis.save(full_path)

In [ ]:
def draw_example():
    vis = Visualizer()
    vis.add_line(((0, 2), (10, 7)), color='black')
    vis.add_point((4, 4), s=30, color='purple')
    vis.add_point((6, 6), s=30, color='green')
    vis.add_point((7, 4), s=30, color='orange')
    vis.show()
    target_dir = pathlib.Path("Imgs/Other")
    target_dir.mkdir(parents=True, exist_ok=True)
    vis.save(target_dir / "example.png")


### Wprowadzenie
Celem ćwiczenia jest określenie po której stronie prostej znajduje się punkt.

In [ ]:
draw_example()

Do tego celu potrzebujesz wygenerować zbiory punktów testowych.


# Generowanie losowych punktów na płaszczyźnie

Uzupełnij funkcję ```generate_uniform_points```
 (Aby generować losowo liczby rzeczywiste bardzo przydatna może okazać się jakaś funckja biblioteczna)

In [ ]:
def generate_uniform_points(left, right, n = 10 ** 5):
    """
    Funkcja generuje równomiernie n punktów na kwadwratowym obszarze od left do right (jednakowo na osi y) o współrzędnych rzeczywistych
    :param left: lewy kraniec przedziału
    :param right: prawy kraniec przedziału
    :param n: ilość generowanych punktów
    :return: tablica punktów w postaci krotek współrzędnych np. [(x1, y1), (x2, y2), ... (xn, yn)]
    """
    return [ tuple( point ) for point in np.random.uniform( left , right , (n,2) ) ]

<span style="color:red">Ćw.</span> Wygeneruj $10^5$ losowych punktów w przestrzeni 2D o współrzędnych z przedziału $x, y \in \left[-1000,1000\right]^{2}$.

In [ ]:
points_a = generate_uniform_points(-1000, 1000, 10 ** 5)

Zwizualizuj wygenerowane punkty

In [ ]:
draw_points( points_a , "1000" )

<span style="color:red">Ćw.</span> Wygeneruj $10^5$ losowych punktów w przestrzeni 2D o współrzędnych z przedziału $ x, y \in \left[-10^{14},10^{14}\right]^{2}$. Sprawdź, czy różni się wizualizalnie wynik tego ćwiczenia z poprzednim.

In [ ]:
points_b = generate_uniform_points(-10 ** 14, 10 ** 14, 10 ** 5)

Zwizualizuj wygenerowane punkty.

In [ ]:
draw_points(points_b, "10_14")

Uzupełnij funkcję ```generate_circle_points```

Zastanów się w sposób generować punkty jednostajnie na okręgu.

In [ ]:
def generate_circle_points(O, R, n = 1000 ):
    """
    Funkcja generuje jednostajnie n punktów na okręgu o środku O i promieniu R
    :param O: krotka współrzędnych x, y określająca środek okręgu
    :param R: promień okręgu 
    :param n: ilość generowanych punktów
    :return: tablica punktów w postaci krotek współrzędnych
    """

    X = [np.random.uniform( 0, 2 * np.pi ) for _ in range( n )]
    points = [( O[0] + R * np.cos(x) , O[1] + R * np.sin(x) ) for x in X]

    return points

<span style="color:red">Ćw.</span> Wygeneruj $ 1000$ losowych punktów w przestrzeni 2D leżących na okręgu o środku $ O = (0,0)$ i promieniu $ R = 100$. 

Uzupełnij funkcję ```generate_points_on_circle_2D```.

In [ ]:
points_c = generate_circle_points((0, 0), 100)

Zwizualizuj wygenerowane punkty.

In [ ]:
draw_points(points_c, "circle_100")

In [ ]:
def generate_collinear_points(a, b, n=100):
    """
    Funkcja generuje równomiernie n współliniowych punktów leżących na prostej ab pomiędzy punktami a i b
    :param a: krotka współrzędnych oznaczająca początek wektora tworzącego prostą
    :param b: krotka współrzędnych oznaczająca koniec wektora tworzącego prostą
    :param n: ilość generowanych punktów
    :return: tablica punktów w postaci krotek współrzędnych
    """
    t_values = np.linspace( 0 , 1 , n )
    vector_ab = np.array(b) - np.array(a)
    points_array = np.array(a) + t_values[: , np.newaxis] * vector_ab

    return [tuple( point ) for point in points_array]



<span style="color:red">Ćw.</span>  Wygeneruj $ 1000$ losowych punktów w przestrzeni 2D o współrzędnej z przedziału $ x \in \langle -1000,1000 \rangle$ leżących na prostej wyznaczonej przez wektor $ \overrightarrow{ab}$. Przyjmij punkty $ a = (-1.0, 0.0)$ oraz $ b = (1.0, 0.1)$. Uzupełnij funkcję ```generate_points_on_line_2D```.

In [ ]:
points_d= generate_collinear_points((-1.0, 0.0), (1.0,0.1))

Zwizualizuj wygenerowane punkty.

In [ ]:
draw_points(points_d,"line_1000")

Przeprowadź test poprawności powyższych funkcji

In [ ]:
Test().runtest(1, generate_uniform_points, generate_circle_points, generate_collinear_points)

# Po której stornie prostej znajduje się punkt?

Prostym sposobem do obliczenia, po której strnie prostej znajduje się punkt jest obliczenie iloczynu wektorowego 
$\overrightarrow{ab} \times \overrightarrow{ac}$, gdzie $ c = (x,y)$ jest punktem, dla którego poszukujemy wiadomości o lokalizacji względem prostej przechodzącej przez punkty $ a$ i $ b$. Metoda ta jest równoznaczna z obliczeniem wyznacznika macierzy $ 2\times2$:  

$$
(1)\det(a, b, c)= \begin{vmatrix}
       a_{x} - c_{x} & a_{y} - c_{y} \\
       b_{x} - c_{x} & b_{y} - c_{y} 
              \end{vmatrix}
$$


lub wyznacznika macierzy $ 3\times3$:

$$
(2)\det(a, b, c)= \begin{vmatrix}
       a_{x} & a_{y} & 1 \\
       b_{x} & b_{y} & 1 \\
       c_{x} & c_{y} & 1
              \end{vmatrix}
$$

Upraszczając tą macierz przez odjęcie drugiego wiersza od trzeciego i odjęcie pierwszego wiersza od drugiego otrzymamy:

$$
\det(a, b, c)  = \begin{vmatrix}
              a_{x}         & a_{y}         & 1 \\
              b_{x} - a_{x} & b_{y} - a_{y} & 0 \\
              c_{x} - b_{x} & c_{y} - b_{y} & 0
                     \end{vmatrix}
              = (b_{x} - a_{x})(c_{y} - b_{y}) - (b_{y} - a_{y})(c_{x} - b_{x})
$$

Jest to wzór, z który opisuje pole równoległoboku mającego boki $ ab$ oraz $ ac$ (Dowód dlaczego tak jest, do zrobienia w domu)  
Dlaczego wiemy, że po obliczeniu wskaźnika podanego powyżej będziemy wiedzieć, po której stornie prostej znajduje się punkt?</font>
<font size="1">
</br>
***

**Dowód**:  

Załóżmy, że mamy dane trzy punkty w przestrzeni 2-wymiarowej $A, B$ oraz $C$. Znajdujemy prostą przechodzącą przez punkty $A$ i $B$. Następnie obliczamy $C_{y}$ przy danym $C_{x}$ i sprawdzamy czy punkt leży nad czy pod prostą.
Współczynnik nachylenia prostej jest nastepujący:

$$a = \frac{B_{y} - A_{y}}{B_{x} - A_{x}}
$$
Natomiast współczynnik $b$ wynosi:

$$b = B_{y} - \frac{(B_{y} - A_{y})B_{x}}{B_{x} - A_{x}}
$$

Po wpisaniu do równania $y = ax + b$ wyliczonego nachylenia prostej, współczynnika $b$ oraz zmiennej $C_{x}$ otrzymujemy:

$$y = \left(\frac{B_{y} - A_{y}}{B_{x} - A_{x}}\right)C_{x}+ \left(B_{y} - \frac{(B_{y} - A_{y})B_{x}}{B_{x} - A_{x}}\right)
$$

Otzymujemy punkt $C$ po lewej stronie prostej jeżeli $C_{y} - y > 0$, po prawej jeżeli $C_{y} - y < 0$, a punkt $C$ leżący na prostej, jeżeli $C_{y} - y = 0$. Przekształcimy powyższe równanie dla $C_{y} - y > 0$:

$$C_{y} - y > 0$$ 
$$C_{y} - \left(\frac{B_{y} - A_{y}}{B_{x} - A_{x}}\right)C_{x} - \left(B_{y} - \frac{(B_{y} - A_{y})B_{x}}{B_{x} - A_{x}}\right) > 0$$
$$C_{y}(B_{x} - A_{x}) - C_{x}(B_{y} - A_{y}) - B_{y}(B_{x} - A_{x}) + B_{x}(B_{y} - A_{y}) > 0$$
$$(C_{y} - B_{y})(B_{x} - A_{x}) + (B_{x} - C_{x})(B_{y} - A_{y}) > 0$$ 
$$(C_{y} - B_{y})(B_{x} - A_{x}) - (C_{x} - B_{x})(B_{y} - A_{y}) > 0$$

Zatem widzimy, że ostatnie równie jest takie same co przy równaniu wyznacznika macierzy $3\times3$. Niejawnie założyliśmy tutaj, że $B_{x}$ jest wieksze od $A_{x}$ , jeżeli byłoby odwrotnie zmieniłby się tylko znak nierówności na przeciwny. W naszym przypadku pokazaliśmy, że $C$ znajduje się po lewej stronie prostej jeżeli wyznacznik jest dodatni oraz po prawej stronie prostej, jeżeli wyznacznik jest ujemny. $Q.E.D$

---
Kolejnym zadaniem będzie zaimplementowanie własnych wyznaczników $(1)$ oraz $(2)$ i porówanie ich szybkości działania z wyznacznikami bibliotecznymi w testowaniu dla różnych zbiorów punktów. Co dodatkowo chcemy sprawdzić, czy wszystkie wyznaczniki podobnie kwalifikują podział względem danej lini.

Uzupełnij funkcje ```mat_det_3x3```

In [ ]:
def mat_det_3x3(a, b, c):
    """
    Obliczanie wyznacznika macierzy 3x3 bez użycia funkcji bibliotecznych
    :param a: krotka współrzędnych (x, y) pierwszego punktu tworzącego naszą prostą
    :param b: krotka współrzędnych (x, y) drugiego punktu tworzącego naszą prostą
    :param c: krotka współrzędnych (x, y) punktu, którego położenie względem prostej chcemy znaleźć
    :return: wartość wyznacznika macierzy
    """
    return ( b[0] - a[0] )*( c[1] - b[1] ) - ( b[1] - a[1] )*( c[0] - b[0] )

Uzupełnij funkcję ```mat_det_3x3_lib```, ale tym razem wykorzystaj dowolną funckję biblioteczną do obliczenia wyznacznika

In [ ]:
def mat_det_3x3_lib(a, b, c):
    """
    Obliczanie wyznacznika macierzy 3x3 z użyciem funkcji bibliotecznych
    :param a: krotka współrzędnych (x, y) pierwszego punktu tworzącego naszą prostą
    :param b: krotka współrzędnych (x, y) drugiego punktu tworzącego naszą prostą
    :param c: krotka współrzędnych (x, y) punktu, którego położenie względem prostej chcemy znaleźć
    :return: wartość wyznacznika macierzy
    """
    M = np.array( [ [a[0],a[1],1],
                    [b[0],b[1],1],
                    [c[0],c[1],1] ] )
    det = np.linalg.det(M)
    return det

Uzupełnij funkcje ```mat_det_2x2```

In [ ]:
def mat_det_2x2(a, b, c):
    """
    Obliczanie wyznacznika macierzy 2x2 bez użycia funkcji bibliotecznych
    :param a: krotka współrzędnych (x, y) pierwszego punktu tworzącego naszą prostą
    :param b: krotka współrzędnych (x, y) drugiego punktu tworzącego naszą prostą
    :param c: krotka współrzędnych (x, y) punktu, którego położenie względem prostej chcemy znaleźć
    :return: wartość wyznacznika macierzy
    """
    return (a[0] - c[0])*(b[1] - c[1]) - (a[1] - c[1])*(b[0] - c[0])

Uzupełnij funkcję ```mat_det_2x2_lib```, ale tym razem wykorzystaj dowolną funckję biblioteczną do obliczenia wyznacznika

In [ ]:
def mat_det_2x2_lib(a, b, c):
    """
    Obliczanie wyznacznika macierzy 2x2 z użyciem funkcji bibliotecznych
    :param a: krotka współrzędnych (x, y) pierwszego punktu tworzącego naszą prostą
    :param b: krotka współrzędnych (x, y) drugiego punktu tworzącego naszą prostą
    :param c: krotka współrzędnych (x, y) punktu, którego położenie względem prostej chcemy znaleźć
    :return: wartość wyznacznika macierzy
    """
    M = np.array( [ [a[0] - c[0], a[1] - c[1]],
                   [b[0] - c[0], b[1] - c[1]] ] )
    det = np.linalg.det(M)
    return det

Przetestujmy napisane powyżej funkcje.

In [ ]:
Test().runtest(2, mat_det_3x3, mat_det_2x2, mat_det_3x3_lib, mat_det_2x2_lib)

<span style="color:red">Ćw.</span> Klasyfikacja punktów względem prostej - zaimplementuj funkcję ```categorize_points```, która skwalifukuje punkty względem prostej wyznacznonej przez wektor $\large \overrightarrow{ab}$ (prosta przechodząca przez punkty $\large a$ oraz $\large b$.

In [ ]:
def categorize_points(points, a, b, mat_det_func, eps):
    """
    :param points: tablica punktów w postaci krotek współrzednych
    :param a: krotka współrzędnych oznaczająca początek odcinka
    :param b: krotka współrzędnych oznaczająca koniec odcinka
    :param mat_det_func: funkcja która będzie tutaj używana do obliczenia wyznacznika macierzy
    :param eps: epsilon - jak blisko wyznacznik macierzy ma być blisko zera, aby uznać punkt za leżący na prostej
    :return: 3 tablice zawierające kolejno zbiory punktów: leżące na lewo od prostej, leżące na prostej, leżące na prawo od prostek
    """
    left = []
    mid = []
    right = []

    for point in points :
        det = mat_det_func( a , b , point )
        if det < -eps :
            right.append( point )
        elif det > eps :
            left.append( point )
        else :
            mid.append( point )
    return left , mid , right

In [ ]:
a = (-1.0, 0.0)
b = (1.0, 0.1)

mat_det_functions = [ mat_det_3x3 , mat_det_3x3_lib , mat_det_2x2 , mat_det_2x2_lib ]
epsilons = [ 0 , 1e-8 , 1e-10 , 1e-12 , 1e-14 ]

Zwizualizuj sklasyfikowane punkty. Punkty różnią się kolorami ze względu na klasyfikację: na lewo od prostej - zielone, na prostej - fioletowe, na prawo - pomarańczowe.

In [ ]:
eps = epsilons[3]
float_type = "float64"

In [ ]:
A_classified_points = [ categorize_points( points_a, a, b, MatDet , eps ) for MatDet in mat_det_functions ]
for i , classified_points  in enumerate( A_classified_points ) :
    title = f"A : {mat_det_functions[i].__name__}, eps = {eps}, {float_type}"
    draw_line( *classified_points , title )

In [ ]:
B_classified_points = [ categorize_points( points_b, a, b, MatDet , eps ) for MatDet in mat_det_functions ]
for i , classified_points  in enumerate( B_classified_points ) :
    title = f"B : {mat_det_functions[i].__name__}, eps = {eps}, {float_type}"
    draw_line( *classified_points , title )

In [ ]:
C_classified_points = [categorize_points(points_c, a, b, MatDet, eps) for MatDet in mat_det_functions]
for i , classified_points  in enumerate( C_classified_points ) :
    title = f"C : {mat_det_functions[i].__name__}, eps = {eps} , {float_type}"
    draw_line( *classified_points , title )

In [ ]:
D_classified_points = [categorize_points(points_d, a, b, MatDet, eps) for MatDet in mat_det_functions]
for i , classified_points  in enumerate( D_classified_points ) :
    title = f"D : {mat_det_functions[i].__name__}, eps = {eps}, {float_type}"
    draw_line( *classified_points , title )

Przeprowadźmy teraz testy dla mniejszej precyzji obliczeń. Do tego celu należy zmiejszyć typ danych z float64 (domyślny typ floata w pythonie) na float32. Różnią się one tym, że float32 jest zapisywane na mniejszej ilości bitów, co przekłada się no mniejszą ilosć cyfr po przecinku.
Zamienić typ floata w całej tablicy można zrobić w następujący sposób:

In [ ]:
float_type = "float32"
points_a_float32 = np.float32(points_a)
points_b_float32 = np.float32(points_b)
points_c_float32 = np.float32(points_c)
points_d_float32 = np.float32(points_d)

In [ ]:
A32_classified_points = [ categorize_points( points_a_float32, a, b, MatDet , eps ) for MatDet in mat_det_functions ]
for i , classified_points  in enumerate( A32_classified_points ) :
    title = f"A32 : {mat_det_functions[i].__name__}, eps = {eps}, {float_type}"
    draw_line( *classified_points , title )


In [ ]:
B32_classified_points = [ categorize_points( points_b_float32, a, b, MatDet , eps ) for MatDet in mat_det_functions ]
for i , classified_points  in enumerate( B32_classified_points ) :
    title = f"B32 : {mat_det_functions[i].__name__}, eps = {eps}, {float_type}"
    draw_line( *classified_points , title )

In [ ]:
C32_classified_points = [categorize_points(points_c_float32, a, b, MatDet, eps) for MatDet in mat_det_functions]
for i , classified_points  in enumerate( C32_classified_points ) :
    title = f"C32 : {mat_det_functions[i].__name__}, eps = {eps} , {float_type}"
    draw_line( *classified_points , title )

In [ ]:
D32_classified_points = [categorize_points(points_d_float32, a, b, MatDet, eps) for MatDet in mat_det_functions]
for i , classified_points  in enumerate( D32_classified_points ) :
    title = f"D32 : {mat_det_functions[i].__name__}, eps = {eps}, {float_type}"
    draw_line( *classified_points , title )

## Eksport wyników do pliku .csv

In [ ]:
classified_points = [
    A_classified_points, B_classified_points, C32_classified_points, D_classified_points,
    A32_classified_points, B32_classified_points, C32_classified_points, D32_classified_points
]
set_labels = ['A', 'B', 'C', 'D', 'A32', 'B32', 'C32', 'D32']

result_list = []

for set_index, label in enumerate(set_labels):
    current_data_set = classified_points[set_index]
    for det_index, det_func in enumerate(mat_det_functions):
        res_tuple = current_data_set[det_index]
        result_list.append( {
            'Typ': label,
            'Epsilon': eps,
            'Wyznacznik': det_func.__name__,
            'Lewo': len(res_tuple[0]),
            'Na Linii': len(res_tuple[1]),
            'Prawo': len(res_tuple[2]),
            } )

final_dataframe = pd.DataFrame(result_list)
final_dataframe.to_csv(f'results_det_classification_eps_{eps}.csv', index=False)

In [ ]:
all_points = [ points_a , points_b , points_c , points_d , points_a_float32 , points_b_float32 , points_c_float32 , points_d_float32 ]
classified_points = [
    A_classified_points, B_classified_points, C32_classified_points, D_classified_points,
    A32_classified_points, B32_classified_points, C32_classified_points, D32_classified_points
]
set_labels = ['A', 'B', 'C', 'D', 'A32', 'B32', 'C32', 'D32']
results_list = []

for eps_index, current_eps_value in enumerate(epsilons):
    for det_index, det_func in enumerate(mat_det_functions):
        for points_set_idx , points_set in enumerate( all_points ) :

            res_tuple = categorize_points(points_set, a, b, det_func, current_eps_value )
            current_label = set_labels[points_set_idx]

            results_list.append( {
                'Typ': current_label,
                'Epsilon': current_eps_value,
                'Wyznacznik': det_func.__name__,
                'Lewo': len(res_tuple[0]),
                'Na Linii': len(res_tuple[1]),
                'Prawo': len(res_tuple[2]),
            } )

final_dataframe = pd.DataFrame(results_list)
final_dataframe.to_csv('final_classification_report_all_epsilons.csv', index=False)